<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/BERT_for_TASC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip Corrected_data.zip


Archive:  Corrected_data.zip
   creating: Corrected_data/
  inflating: Corrected_data/BombBlast(categorised).json  
  inflating: Corrected_data/Earthquake(categorised).json  
  inflating: Corrected_data/flood(categorised).json  
  inflating: Corrected_data/tornado(categorised).json  
  inflating: Corrected_data/tsnuami (categorised).json  
  inflating: Corrected_data/Volcano(categorised).json  
  inflating: Corrected_data/war(categorised).json  


In [2]:
import json
import os

all_data = []
for filename in os.listdir('Corrected_data/'):
    if filename.endswith(".json"):
        file_path = os.path.join('Corrected_data/', filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            all_data.extend(data)

with open("Data.json", 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
import copy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:

data=pd.read_json("Data.json")

In [5]:
data.head()

,text,intent
0,The officers then violently yanked Ms. Young a...,Miscellaneous
1,"On Tuesday, it was announced that the ""Full Ho...",Miscellaneous
2,‘We are walking a very tight line between tack...,Miscellaneous
3,Warning: Graphic content Death threat to Lenno...,Micscellaneous
4,JUST WATCHEDObama warns the foundation of demo...,Micsellaneous


In [6]:
data["intent"].unique()

array(['Miscellaneous', 'Micscellaneous', 'Micsellaneous', 'War',
       'Micellaneous', 'miscellaneous', 'Miscellaeous', 'Volcano',
       'Tornado', 'Miscellanous', 'Miscellenous', 'Miscelleneous',
       'BombBlast', 'Earthquake', nan, 'floods', 'Miscellaenous',
       'Miscellneous'], dtype=object)

In [7]:
# checlking for nan value
data['intent'].fillna('misc', inplace=True)

data.loc[data["intent"].str.startswith("m"),"intent"]="misc"
data.loc[data["intent"].str.startswith("M"),"intent"]="misc"

In [8]:
data["intent"].unique()

array(['misc', 'War', 'Volcano', 'Tornado', 'BombBlast', 'Earthquake',
       'floods'], dtype=object)

In [9]:
import nltk
nltk.download('stopwords') # for removing useless data
nltk.download('punkt') # use unsupervised algorithm to build a model for abbreviation words, collocations, and words
nltk.download('wordnet') #used  in information systems, word-sense disambiguation, information retrieval, automatic text classification, summarization, machine translation.
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [10]:
# clean the function will apply the above mentioned preprocessing to the resumes
def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    # make list of text into string
    result=" ".join(clean_text)
    return result

In [11]:
!pip install transformers
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00


In [12]:
# from transformers import BertTokenizer, TFBertModel
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf

In [13]:
model_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name) #hugging face bert tokenizer
model = AutoModel.from_pretrained(model_name) #hugging face bert model for embedings


In [14]:
def token_and_vector(features, batch_size=16):
    clean_texts = [clean(text) for text in features]
    vectors = []
    for i in range(0, len(clean_texts), batch_size):
        batch = clean_texts[i:i+batch_size]
        texts_tokens= tokenizer.batch_encode_plus(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = model(**texts_tokens)
            vectors_batch = outputs.last_hidden_state.mean(dim=1).numpy()
        vectors.append(vectors_batch)
    return np.concatenate(vectors, axis=0)



In [15]:
features = token_and_vector(data["text"])

In [16]:
x_train, x_test, y_train, y_test = train_test_split(features,data["intent"],test_size=0.2, random_state=42)
len(x_train),len(x_test)

(390, 98)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

rfc = RandomForestClassifier()   # Random Forest
mbc = MultinomialNB(force_alpha=True) # MultinomialNB
svc = SVC(kernel='linear', C=1.0) # Support Vector Machine
lrc=LogisticRegression() # Logistic regression




rfc.fit(x_train,y_train)
mbc.fit(scaler.fit_transform(x_train),y_train)
svc.fit(x_train,y_train)
lrc.fit(x_train,y_train)

# traning using the 70% data we have extracted from the dataset
# scores = cross_val_score(rf,X_train,y_train,cv=5)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
models=[ rfc,mbc,svc,lrc]
models_name=["Random Forest","MultinomialNB","Support Vector Machine","Logistic regression"]
models_list={}
for i,j in zip(models_name,models):
  models_list[i]=j
  print(i,j)


Random Forest RandomForestClassifier()
MultinomialNB MultinomialNB(force_alpha=True)
Support Vector Machine SVC(kernel='linear')
Logistic regression LogisticRegression()


In [19]:
models_list

{'Random Forest': RandomForestClassifier(),
 'MultinomialNB': MultinomialNB(force_alpha=True),
 'Support Vector Machine': SVC(kernel='linear'),
 'Logistic regression': LogisticRegression()}

In [20]:
from sklearn.metrics import f1_score

def accuracy_score(models,X_test,y_test):
  f1_scores={}
  for model in models.keys():
    pred=models[model].predict(X_test)
    score=f1_score(y_test,pred,average='micro')
    f1_scores[model]=score
  return f1_scores



In [21]:
f1_scores=accuracy_score(models_list,x_test,y_test)
print(f1_scores)


{'Random Forest': 0.7346938775510203, 'MultinomialNB': 0.6224489795918368, 'Support Vector Machine': 0.806122448979592, 'Logistic regression': 0.806122448979592}


In [22]:
import pickle

# Save the trained model (e.g., Random Forest classifier)
with open('model.pkl', 'wb') as model_file:
    pickle.dump(models_list['Support Vector Machine'], model_file)

# Save the BERT vectorizer (tokenizer and model)
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump((tokenizer, model), vectorizer_file)


In [23]:
import pickle

# Load the trained model
with open('model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Load the BERT vectorizer (tokenizer and model)
with open('vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_tokenizer, loaded_bert_model = pickle.load(vectorizer_file)


In [24]:
# Preprocess new text data (similar to what you did before)
article = "A man who lost £450 to a fake landlord in a rental scam has urged others to be careful looking for properties online. Craig Barton gave the money to a man he thought was the landlord of an apartment in Belfast city centre. The PSNI say they have received five reports of people falling victim to that one scam. Experts say a shortage of private rental accommodation, and rising prices, are creating perfect conditions for fraudsters to thrive. \"You're basically being targeted because you're vulnerable and you're wanting this house so much,\" said Craig. The 29-year-old had been looking for somewhere to rent for nearly two years. He doesn't have a guarantor, which makes the process more difficult. So when he saw a room online in a Belfast city centre apartment, he set up a viewing. \"I met the person, they seemed really nice. He took me in, showed me about the house, showed me how to use all the heating and everything,\" he said. \"There was someone in the room I was going to live in and he said he was throwing him out for being too loud.\" When Craig, a chef, returned, the man had a contract ready to sign. \"That next day when I went in, he had a contract on his phone, it looked real and everything, all written out. It had his signature on it and I put my signature on it. \"He gave me the key to the door, just to the apartment. When I left, I said I didn't have a key for the main door of the building, he said you'll get it when the other tenant moves out.\" Craig handed over £450 as a deposit. A few days later when he arrived to move in, he tried calling the man he'd been texting. \"There was no reply so I was getting the feeling that something was going wrong. I couldn't get in the main door, so I rang the buzzer. \"It was someone else that answered, they said they didn't know what I was talking about. The number [of the apartment] he gave me was wrong. \"I was gutted after, I felt like I lost all my money and everything.\" Amy Lyness of Housing Rights, a group which offers housing advice in the social and private sectors, said they are hearing about a lack of available housing across the board. \"They [fraudsters] would understand that people are desperate, that there's a lack of available housing along with the increased prices that people are having to pay,\" she said. \"They can produce an advertisement that says there's available housing at a slightly reduced rent. \"They're going on the basis that desperate people will want to jump on that good deal and that's where they're likely to get caught without being able to establish the legitimacy of the tenancy.\" The Police Service of Northern Ireland (PSNI) said it was investigating Craig's report and four others related to the same scam. Belfast District Supt Gavin Kirkpatrick is urging people to be vigilant. \"With so many people seeking properties online, scammers will look for opportunities to take advantage and swindle people out of their money,\" he said. \"So, it's really important to be vigilant and take some time to do your research on the company, or person you are dealing with. \"Confirm the advertiser is genuine, and check the property exists and is available. If it's a scam there's a good chance others will write about their experience and some basic internet research will find it.\" Figures from Action Fraud, the national fraud reporting centre, show there were 5,751 reports of rental scams across the UK in 2022, up nearly a quarter on the previous year. One popular site Gumtree recently issued a warning to its users. Craig found the apartment listed initially on a Facebook group. A spokesperson for Meta, Facebook's parent company, said this was an \"industry-wide issue\". \"Scammers are using increasingly sophisticated methods to defraud people in a range of ways including email, SMS and offline,\" she said. They added the company spent $5bn (£4.08bn) last year on safety and security and said they encourage people to report activity like this to Meta and the police."
cleaned_article = clean(article)

# Tokenize and obtain BERT embeddings using the loaded vectorizer
article_tokens = loaded_tokenizer.batch_encode_plus([cleaned_article], padding=True, truncation=True, return_tensors='pt', max_length=512)
with torch.no_grad():
    outputs = loaded_bert_model(**article_tokens)
    vector = outputs.last_hidden_state.mean(dim=1).numpy()

# Use the loaded model for prediction
prediction = loaded_model.predict(vector)

print(f"Predicted intent: {prediction[0]}")


Predicted intent: misc
